In [7]:
import pandas as pd
import numpy as np 
from pandarallel import pandarallel

In [8]:
import sklearn

In [ ]:
pandarallel.initialize()


# Data Examination and Preparation #

In [9]:
dataset = pd.read_csv('GTEX_data.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [10]:
dataset_sorted_id = dataset.sort_values(by=['Unnamed: 0'])
del dataset

Dataset is sorted so that the test and training set won't have the same person's entries in each, in order to prevent data leakage. 

Note that this method is not perfect and will allow one person's entries to appear in both test and training set. 

In [ ]:
dataset_sorted_id.isnull().sum().sum()

No NANs in dataset. 


In [ ]:
#non stratified split method

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


#label encode targets
le = LabelEncoder()
dataset_sorted_id["tissue"] = le.fit_transform(dataset_sorted_id["tissue"])

#split up, and use if statement to ensure patients don't appear in different sets
train_full, test = train_test_split(dataset_sorted_id, test_size=0.2, shuffle=False)
if train_full['Unnamed: 0'].iloc[-1] == test['Unnamed: 0'].iloc[0]:
    rows = test.loc[test['Unnamed: 0'] == test['Unnamed: 0'].iloc[0]]
    train_full = train_full.append(rows, ignore_index=True)
    test.drop(rows.index, inplace=True)
    
    
train, val = train_test_split(train_full, test_size=0.2, shuffle=False)
if train['Unnamed: 0'].iloc[-1] == val['Unnamed: 0'].iloc[0]:
    rows = val.loc[val['Unnamed: 0'] == val['Unnamed: 0'].iloc[0]]
    train = train.append(rows, ignore_index=True)
    val.drop(rows.index, inplace=True)


X_train = train.drop(["tissue", "Unnamed: 0"], axis=1)
X_test = test.drop(["tissue", "Unnamed: 0"], axis=1)
X_val = val.drop(["tissue", "Unnamed: 0"], axis=1)

y_train = (train["tissue"])
y_test = (test["tissue"])
y_val = (val["tissue"])

#scale all features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

del train_full, test, train, val, X_train, X_test, X_val


In [ ]:
#stratified split method with enforcing of patients not being in different classes

#below stratifies by tissue by making new column "tr" and labelling True if that row should be in the train set
for tissue in dataset_sorted_id["tissue"].unique():
    row_nums = dataset_sorted_id[dataset_sorted_id["tissue"]==tissue].index
    all_cat_idx = np.array(row_nums)
    n_tr = np.ceil(len(row_nums)*0.8)
    indices = np.random.choice(all_cat_idx, int(n_tr), replace=False)
    dataset_sorted_id.loc[indices, "tr"] = True
    
dataset_sorted_id.loc[dataset_sorted_id["tr"] != True, "tr"] = False

#below code finds the unique patients in tr and the unique ones in te
tr_patients = np.array(dataset_sorted_id.loc[dataset_sorted_id['tr'] == True, 'Unnamed: 0'].unique())
te_patients = np.array(dataset_sorted_id.loc[dataset_sorted_id['tr'] == False, 'Unnamed: 0'].unique())

intersection = list(set(tr_patients) & set(te_patients))
#the intersection is of size 792. There are 838 patients in total. We would then have to randomly move 80% of these 792 patients
#train set and 20% to test set. We would basically lose our formal stratification then. 

In [ ]:
number_classes = len(le.classes_)
number_classes

## Check stratification ##

In [ ]:
def compare_stratification(subset1, subset2, dataset=dataset):
    """A function to compare the stratification in two subsets of the dataset (i.e. test and val) to that in the whole dataset.
    Dataset is the whole dataset.
    Subset are subsets, which are here are Series since we will be passing in the features 'y' as we want 
    stratification on the tissues.
    Creates a table showing the % difference in stratification"""
    
    def tissue_proportions(data):
        return data["tissue"].value_counts()/len(data)
    
    def tissue_proportions_sub(data):
        return data.value_counts()/len(data)
    
    compare_props = pd.DataFrame({
        "Overall": tissue_proportions(dataset),
        "Subset1": tissue_proportions_sub(subset1),
        "Subset2": tissue_proportions_sub(subset2),
            }).sort_values(by='Overall', ascending=False)
    compare_props["subset1. %error"] = 100 * tissue_proportions_sub(subset1)/ tissue_proportions(dataset) - 100
    compare_props["subset2. %error"] = 100 * tissue_proportions_sub(subset2) / tissue_proportions(dataset) - 100
    
    return compare_props

In [ ]:
compare_stratification(subset1=y_test, subset2=y_val, dataset=dataset_sorted_id)

# Model Exploration #

## Random Forest ##

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, verbose=3)

In [ ]:
rf.fit(X_train_scaled, y_train)

In [ ]:
rf.score(X_train_scaled, y_train)

Can fit to training set. Poor result on test set -> overfitting. Can examine with hyperparameter tuning later.  

In [ ]:
rf.score(X_test_scaled, y_test)

Result is 0.4

### Hyperparameter tuning ###

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

In [ ]:
parameter_grid = {

  'n_estimators': sp_randint(200,600),
  'max_depth':  sp_randint(20,110),
}

In [ ]:
rnd_search = RandomForestClassifier(n_jobs=-1, verbose=3)
random_search_rf = RandomizedSearchCV(rnd_search, parameter_grid, cv=KFold(n_splits=3, shuffle=False), random_state=42, verbose=3, n_iter=5)

random_search_rf.fit(X_train_scaled,y_train)

result = pd.concat([pd.DataFrame(random_search_rf.cv_results_["params"]),pd.DataFrame(random_search_rf.cv_results_["mean_test_score"], columns=["Score"])],axis=1)
result.sort_values(by="Score", ascending=False, inplace=True)

result

Best score is around 0.57. So RF can overfit well, but struggling to generalise. Would need more estimators for a better score but it doesn't look to be as promising as the NN. Why? I thought RF should be comparable given it is tabular data? 

In [ ]:
rf= RandomForestClassifier(n_jobs=-1, verbose=3, max_depth=71, n_estimators=548)

In [ ]:
rf.fit(X_train_scaled, y_train)

In [ ]:
rf.score(X_val_scaled, y_val)

In [ ]:
rf.score(X_test_scaled, y_test)

Test and val scores are not too disimilar (unlike NN) -> maybe NN just performs far better on val set - val set is different in composition to test set after all.

## Neural network ##

In [ ]:
import tensorflow as tf
from tensorflow import keras 

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
tf.__version__


In [ ]:
keras.__version__

Only testing on one validation set is not robust right? You could do some sort of cross validation where you select different validation folds, but then training is just longer. I guess it depends on the size of the validation fold. But what is done in practice? -> just one val set.

In [ ]:

X_valid, X_train = X_val_scaled, X_train_scaled
y_valid, y_train = y_val, y_train

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(1000, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dropout(rate = 0.2)
    keras.layers.Dense(number_classes, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

#callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1.5)
plt.show()

In [ ]:
model.evaluate(X_test_scaled, y_test)

In [ ]:
##sample##

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()


tf.nn.softmax(predictions).numpy()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


In [ ]:
tf.config.experimental.list_physical_devices('GPU')